# 스팀 게임목록 크롤링

## 조건
- 대상 : Steam 멀티플레이어 카테고리 전체 게임 목록
- 페이지 offset 기반, 12개 단위로 수집
- 언어 : 한국어
- 중복제거 : 게임명+출시일 기준
- 중간저장 : 1000개씩

## 절차
- offset값 증가시키며 목록 페이지 순회
- 각 페이지에서 게임명, 출시일, 장르, os지원, 평가점수, 평가 수 추출
- 이미 수집된 게임은 스킵
- 1000개 단위로 중복 저장, 크롤링 종료 시 최종 csv 생성
- 임시파일 삭제 및 크롤링 페이지 종료
    * 중간에 크롤링 중단 시 offset 값과 최종 파일명을 변경하여 재개

In [ ]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Selenium 설정
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 유틸 함수
def safe_find(card, selector, attr="text"):
    try:
        el = card.find_element(By.CSS_SELECTOR, selector)
        return el.text.strip() if attr == "text" else el.get_attribute(attr)
    except:
        return ""

def extract_game_data(card):
    try:
        name = card.find_element(By.CLASS_NAME, "_2ekpT6PjwtcFaT4jLQehUK").text.strip()
    except:
        return None

    try:
        release = card.find_element(By.CLASS_NAME, "_1qvTFgmehUzbdYM9cw0eS7").text.strip()
    except:
        release = ""

    genre_tags = card.find_elements(By.CSS_SELECTOR, 'a._33yqka47vWurNqhnhLJb_m.WidgetTag')
    genre = " | ".join(tag.text.strip() for tag in genre_tags if tag.text.strip())

    return {
        "게임명": name,
        "출시일": release,
        "장르": genre,
        "OS": safe_find(card, 'span._2bCf9u4rlC8De687HY6wnh span[title="Windows"]', "title"),
        "평가점수": safe_find(card, 'div._2nuoOi5kC2aUI12z85PneA', "aria-label"),
        "사용자 평가 수": safe_find(card, 'div._1wXL_MfRpdKQ3wZiNP5lrH', "aria-label")
    }

def save_to_csv(data, filename, is_temp=False):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"{'중간 저장' if is_temp else '최종 저장'}: {filename} ({len(data)}개)")

# 크롤링 시작
results = []
seen_keys = set()
offset = 42568
step = 12
batch_count = 0

print("크롤링 시작...")

while True:
    url = f"https://store.steampowered.com/category/multiplayer/?flavor=contenthub_all&offset={offset}&l=korean"
    print(f"페이지 요청 중: offset={offset}")
    driver.get(url)
    time.sleep(3)  # 로딩 대기

    try:
        cards = driver.find_elements(By.CLASS_NAME, "_1_P15GG6AKyF_NMX2j4-Mu")
    except:
        print("카드 요소를 찾을 수 없음, 종료")
        break

    if not cards:
        print("더 이상 게임 없음, 종료")
        break

    new_items = 0
    for card in cards:
        data = extract_game_data(card)
        if data:
            key = f"{data['게임명']}_{data['출시일']}"
            if key not in seen_keys:
                seen_keys.add(key)
                results.append(data)
                new_items += 1

    print(f"{new_items}개 수집 완료 (총 {len(results)}개)")

    # 중간 저장 (1000개 단위)
    if len(results) // 1000 > batch_count:
        batch_count = len(results) // 1000
        save_to_csv(results, f"steam_batch_{batch_count}.csv", is_temp=True)

    if new_items == 0:
        print("새로운 항목 없음, 종료")
        break

    offset += step
    time.sleep(1)  # 서버 부담 방지

# 최종 저장
final_filename8 = "steam_multiplayer_full8.csv" # 여기 수정
save_to_csv(results, final_filename8) # 여기 수정

# 임시 파일 정리
for i in range(1, batch_count + 1):
    temp_file = f"steam_batch_{i}.csv"
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"🧹 {temp_file} 삭제 완료")

driver.quit()
print(f"\n🎉 크롤링 완료! 총 {len(results)}개 저장됨: {final_filename8}") # 여기 수정

🚀 크롤링 시작...
📥 페이지 요청 중: offset=42568
✅ 12개 수집 완료 (총 12개)
📥 페이지 요청 중: offset=42580
✅ 12개 수집 완료 (총 24개)
📥 페이지 요청 중: offset=42592
✅ 12개 수집 완료 (총 36개)
📥 페이지 요청 중: offset=42604
✅ 12개 수집 완료 (총 48개)
📥 페이지 요청 중: offset=42616
✅ 12개 수집 완료 (총 60개)
📥 페이지 요청 중: offset=42628
✅ 12개 수집 완료 (총 72개)
📥 페이지 요청 중: offset=42640
✅ 12개 수집 완료 (총 84개)
📥 페이지 요청 중: offset=42652
✅ 12개 수집 완료 (총 96개)
📥 페이지 요청 중: offset=42664
✅ 12개 수집 완료 (총 108개)
📥 페이지 요청 중: offset=42676
✅ 12개 수집 완료 (총 120개)
📥 페이지 요청 중: offset=42688
✅ 12개 수집 완료 (총 132개)
📥 페이지 요청 중: offset=42700
✅ 12개 수집 완료 (총 144개)
📥 페이지 요청 중: offset=42712
✅ 12개 수집 완료 (총 156개)
📥 페이지 요청 중: offset=42724
✅ 7개 수집 완료 (총 163개)
📥 페이지 요청 중: offset=42736
✅ 12개 수집 완료 (총 175개)
📥 페이지 요청 중: offset=42748
✅ 12개 수집 완료 (총 187개)
📥 페이지 요청 중: offset=42760
✅ 12개 수집 완료 (총 199개)
📥 페이지 요청 중: offset=42772
✅ 9개 수집 완료 (총 208개)
📥 페이지 요청 중: offset=42784
✅ 11개 수집 완료 (총 219개)
📥 페이지 요청 중: offset=42796
✅ 12개 수집 완료 (총 231개)
📥 페이지 요청 중: offset=42808
✅ 12개 수집 완료 (총 243개)
📥 페이지 요청 중: offset=42820
✅ 12개 수